In [1]:
import os

In [2]:
%pwd


'c:\\Users\\azfar\\Downloads\\wine-ml-proj\\wine-ml\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd


'c:\\Users\\azfar\\Downloads\\wine-ml-proj\\wine-ml'

In [5]:
from wineml.constants import *

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [7]:
from wineml.constants import *
from wineml.utils.common import read_yaml, create_directories

In [8]:
from pathlib import Path
from box import ConfigBox
from wineml.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from wineml.utils.common import read_yaml, create_directories
from wineml.entity.config_entity import (
    DataIngestionConfig,
    DataValidationConfig,
    DataTransformationConfig,
    ModelTrainerConfig,
)

class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
        schema_filepath: Path = SCHEMA_FILE_PATH
    ):
        self.config: ConfigBox = read_yaml(config_filepath)
        self.params: ConfigBox = read_yaml(params_filepath)

        # ✅ Robust safe check for schema.yaml
        if Path(schema_filepath).exists():
            self.schema: ConfigBox = read_yaml(schema_filepath)
        else:
            self.schema = None

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN if self.schema else {"name": "quality"}

        create_directories([Path(config.root_dir)])

        model_trainer_config = ModelTrainerConfig(
        root_dir=config.root_dir,
        train_data_path=config.train_data_path,
        test_data_path=config.test_data_path,
        model_name=config.model_name,
        alpha=params.alpha,
        l1_ratio=params.l1_ratio,
        target_column=schema["name"] if isinstance(schema, dict) else schema.name
    )

        return model_trainer_config


In [9]:
import pandas as pd
import os
from wineml.logging.logger import logger
from sklearn.linear_model import ElasticNet
import joblib

In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
        

In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()

except Exception as e:
    logger.exception(e)
    raise e

[2025-05-07 14:21:31,992: INFO]: yaml file: C:\Users\azfar\Downloads\wine-ml-proj\wine-ml\config\config.yaml loaded successfully


[2025-05-07 14:21:31,992]: INFO: common: yaml file: C:\Users\azfar\Downloads\wine-ml-proj\wine-ml\config\config.yaml loaded successfully]


[2025-05-07 14:21:31,995: INFO]: yaml file: C:\Users\azfar\Downloads\wine-ml-proj\wine-ml\params.yaml loaded successfully


[2025-05-07 14:21:31,995]: INFO: common: yaml file: C:\Users\azfar\Downloads\wine-ml-proj\wine-ml\params.yaml loaded successfully]


[2025-05-07 14:21:31,998: INFO]: created directory at: artifacts


[2025-05-07 14:21:31,998]: INFO: common: created directory at: artifacts]


[2025-05-07 14:21:31,999: INFO]: created directory at: artifacts\model_trainer


[2025-05-07 14:21:31,999]: INFO: common: created directory at: artifacts\model_trainer]
